In [39]:
import json
import pandas as pd
import statistics

spiList = [5.55, 5.72, 7.3, 7.75, 8.4, 9, 8.8, 8.2]
print(statistics.mean(spiList))

import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import operator


7.59


In [23]:
from gensim.models.keyedvectors import FastTextKeyedVectors
fasttext = FastTextKeyedVectors.load("D:/fasttext_word2vec/araneum_none_fasttextcbow_300_5_2018/araneum_none_fasttextcbow_300_5_2018.model")


In [ ]:
НЕПОНЯТНЫЕ ПРБЛЕМЫ С ВЕКТОРОМ ЧАСТОТНОСТЕЙ

In [37]:
with open ("lyashevskaya_freq_dict.json" , "r", encoding="utf-8") as f:
    lyashevskaya_freq_dict = json.load(f)
    
with open ("C:\Autotutor\improved_approach\colloc\music_unigr_freq.json", "r", encoding='utf-8') as f:
    colloc_db = json.load(f)
    
    
with open ("D:\input\music_smart_colloc_freq.json", "r", encoding='utf-8') as f:
    unigramm_db = json.load(f)
    
words_indexed_dict = []
with open("big_musiciancolloc_lit.txt", "r", encoding = "utf-8") as f:
    for line in f.readlines():
        words_indexed_dict.append(line[:-1])

In [10]:
USER_NAME = 'big_musician'

In [13]:

marked_word_db = pd.read_csv(USER_NAME +  "_word_db.csv",header = None)

In [29]:
def get_vector(ngramm):
    ngramm_lemmas_list = ngramm.split()
    freq_list = []
    local_corpora_freq_list = []
    w2v_list = []
    for lemma in ngramm_lemmas_list:
        try:
            w2v = fasttext[lemma]
            w2v_list.append(w2v)
        except:
            pass
        if lemma in lyashevskaya_freq_dict:
            freq_list.append(lyashevskaya_freq_dict[lemma])
        else:
            freq_list.append(0)
        if lemma in unigramm_db:
            local_corpora_freq_list.append(unigramm_db[lemma])
        else:
            local_corpora_freq_list.append(0)
    colleted_w2v_count = len(w2v_list)
    if colleted_w2v_count > 0:
        vect_sum = 300 * [0]
        for w2v in w2v_list:
            vect_sum += w2v
        vect_sum /=  colleted_w2v_count
    else:
        print(ngramm_lemmas_list, "none in fasttext")
        vect_sum = None
    freq_mean = statistics.mean(freq_list)
    local_corpora_mean = statistics.mean(local_corpora_freq_list)
    
    return (local_corpora_mean,freq_mean, vect_sum)

In [51]:
list_of_all_words_similarity = []
with open("big_musiciancolloc_lit.txt", 'r', encoding = 'utf-8') as f:
    for colloc_word in f.readlines():
        current_word_similarity = []
        #print(colloc[1][0])
        colloc_word_vector = get_vector(colloc_word[:-1])
        freq_vec = np.array([colloc_word_vector[0], colloc_word_vector[1]]).reshape(1,-1)
        colloc_w2v = np.array(colloc_word_vector[2]).reshape(1,-1)
        for db_word_index in range(len(marked_word_db)):
            compared_raw_word_vect = marked_word_db.iloc[db_word_index][:-1]

            comp_word2vec = compared_raw_word_vect[2:]
            #print(db_word_index, len(comp_word2vec))
            comp_colloc_w2v = np.array(comp_word2vec).reshape(1,-1)
            w2v_sim = cosine_similarity(colloc_w2v,comp_colloc_w2v)
            #print("w2v", sim)


            comp_freq_vec = np.array([compared_raw_word_vect[1], compared_raw_word_vect[0]]).reshape(1,-1)
            sim_freq = cosine_similarity(freq_vec,comp_freq_vec)
            #print("freq", sim_freq)

            w2v_sim = w2v_sim[0][0]
            sim_freq = sim_freq[0][0]

            average_similarity = w2v_sim*0.8 + sim_freq*0.2

            word_comp_json = {"colloc_db_index":db_word_index, "sim_colloc":words_indexed_dict[db_word_index],"w2v_sim":w2v_sim,"freqence_similarity":sim_freq}
            if average_similarity> 0.6 and w2v_sim > 0.65:
                current_word_similarity.append((word_comp_json,average_similarity))
        current_word_similarity.sort(key = operator.itemgetter(1), reverse= True)
        list_of_all_words_similarity.append((colloc_word[:-1],current_word_similarity[:5]))
        break

In [52]:
with open ("big_musician_colloc_similairities.json", 'w') as f:
    json.dump(list_of_all_words_similarity, f , indent = 4, ensure_ascii=False)